In [40]:
import json
import csv
import os

In [41]:
# establishes CSV

headers = ["age", "language", "smoking_frequency", 
               "never_smoked", "smoked_at_some_point", "currently_smoking"]
    
with open('b2ai_smoking_data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(headers)

In [42]:
def pullInfo(file, target):
    if file == None:
        return None
    with open(file, 'r') as f:
        data = json.load(f)
    value = None
    for item in data.get('item', []):
        if item.get('linkId') == target:
            answer = item.get('answer', [{}])[0]
            if 'valueBoolean' in answer:
                value = answer['valueBoolean']
            elif 'valueString' in answer:
                value = answer['valueString']
            break

    return value  

def addRow(data):
    with open('b2ai_smoking_data.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(data)

def getPartSurv(sub):
    for surv in os.listdir(sub):
        if surv[-9:-5] == "pant":
            return os.path.join(sub, surv)

def getConSurv(sub):
    for ses in os.listdir(sub):
        if ses[0:3] == 'ses':
            x = os.path.join(sub, ses)
            for beh in os.listdir(x):
                if beh == 'beh':
                    y = os.path.join(x, beh)
                    for f in os.listdir(y):
                        if f[-30:-5] == 'qgenericconfoundersschema':
                            t = os.path.join(y, f)
                            return t
    


In [43]:
#Establishes Disctionary

d = dict()
for sub in os.listdir('bids_with_sensitive_recordings'):
    if sub[0:3] == 'sub':
        i = os.path.join('bids_with_sensitive_recordings', sub)
        d[i] = []


In [44]:
#checks possible responces to questions of interest

def printResp(ques, surv):
    s = set()
    for i in d:
        if surv == 'part':
            
            s.add(pullInfo(getPartSurv(i), ques))
        elif surv == 'cons':
            s.add(pullInfo(getConSurv(i), ques))
    print(f'{ques} : {s} \n')


printResp('age', 'part')
printResp('selected_language', 'part')
printResp('smoking_freq', 'cons')
printResp('smoking_hx', 'cons')



age : {'43.0', '51.0', '33.0', '65.0', '79.0', '25.0', '38.0', '68.0', '55.0', '37.0', '59.0', '54.0', '27.0', '30.0', '63.0', '31.0', '45.0', '90.0', '76.0', '21.0', '20.0', '60.0', '26.0', '73.0', '80.0', '75.0', '41.0', '69.0', '82.0', '48.0', '70.0', '67.0', '23.0', '57.0', '83.0', '61.0', '44.0', '42.0', '46.0', '29.0', '39.0', '62.0', '50.0', '32.0', '53.0', '24.0', '58.0', '74.0', '84.0', '19.0', '52.0', '71.0', '22.0', '40.0', '28.0', '66.0', '36.0', '72.0', '56.0', '64.0', '93.0'} 

selected_language : {'English'} 

smoking_freq : {'A few times a week', None, 'A few times a year', 'About once a day', 'Multiple times a day', 'A few times a month'} 

smoking_hx : {'I used to smoke', None, 'Prefer not to answer', "I've never smoked regularly", 'I currently smoke'} 



In [45]:
#Fills Dictionary

for i in d:

    d[i].append(pullInfo(getPartSurv(i), 'age'))
    d[i].append('en')
    smokF =  pullInfo(getConSurv(i), 'smoking_freq')
    if (smokF == 'A few times a week') | (smokF == 'A few times a year') | (smokF == 'A few times a month') : 
        d[i].append('less-than-daily')
    elif smokF == None:
        d[i].append('not-at-all')
    else:
        d[i].append('daily')
    smokH = pullInfo(getConSurv(i), 'smoking_hx')
    if smokH == 'I used to smoke':
        d[i].append(False)
        d[i].append(True)
        d[i].append(0)
    elif smokH == 'I currently smoke':
        d[i].append(False)
        d[i].append(True)
        d[i].append(1)
    else:
        d[i].append(True)
        d[i].append(False)
        d[i].append(0)



In [46]:
#Sanity checks Dictionary

def check():
    for i in d:
        p = d[i]
        if p[2] == 'not-at-all':
            if (p[3] != True) & (p[4] != False) & (p[5] != 0):
                return False
        else:
            if (p[3] == True) | (p[4] == False):
                return False
    return True
check()

True

In [47]:
#adds Data to the csv

for i in d:
    addRow(d[i])